# Coursework 2: Data Processing

## Task 1
This coursework will assess your understanding of using NoSQL to store and retrieve data.  You will perform operations on data from the Enron email dataset in a MongoDB database, and write a report detailing the suitability of different types of databases for data science applications.  You will be required to run code to answer the given questions in the Jupyter notebook provided, and write a report describing alternative approaches to using MongoDB.

Download the JSON version of the Enron data (using the “Download as zip” to download the data file from http://edshare.soton.ac.uk/19548/, the file is about 380MB) and import into a collection called messages in a database called enron.  You do not need to set up any authentication.  In the Jupyter notebook provided, perform the following tasks, using the Python PyMongo library.

Answers should be efficient in terms of speed.  Answers which are less efficient will not get full marks.

In [40]:
import pymongo
from pymongo import MongoClient
from datetime import datetime
from pprint import pprint

"""
Versions in this assignment
MongoDB: 4.2.1
Python: 2.7.17
Pymongo: 3.9.0
"""

'\nVersions in this assignment\nMongoDB: 4.2.1\nPython: 2.7.17\nPymongo: 3.9.0\n'

### 1)
Write a function which returns a MongoDB connection object to the "messages" collection. [4 points] 

In [41]:
def get_collection():
    """
    Connects to the server, and returns a collection object
    of the `messages` collection in the `enron` database
    """
    # YOUR CODE HERE
    client = MongoClient('mongodb://localhost:27017')
    return client.enron.messages

### 2)

Write a function which returns the amount of emails in the messages collection in total. [4 points] 

In [42]:
def get_amount_of_messages(collection):
    """
    :param collection A PyMongo collection object
    :return the amount of documents in the collection
    """    
    # YOUR CODE HERE
    return collection.count_documents({})

### 3) 

Write a function which returns each person who was BCCed on an email.  Include each person only once, and display only their name according to the X-To header. [4 points] 



In [44]:
import re
# 2289

def get_bcced_people(collection):
    """
    :param collection A PyMongo collection object
    :return the names of the people who have received an email by BCC
    """    
    # YOUR CODE HERE    
    # Find all distinct names in the headers.X-bcc
    BCC = collection.distinct('headers.X-bcc')
    
    # regex pattern to extract emails or names in headers.X-bcc
    pattern = r'(?<!<)\b\S+@\S+\.\S+\b(?!>)|\b\w[^<>]+(?=\s<.+?>)'
    names = [name.strip() for items in BCC for name in re.findall(pattern, items)]

    return list(set(names))
    
#     # Find emails that Bcc not empty and Bcc is equal to Cc
#     match = {
#         '$match': {'$and': [{'headers.Bcc': {'$ne': None}},
#                             {'$expr': {'$eq': ['$headers.Bcc', '$headers.Cc']}}
#                            ]}
#     }
#     project = {
#         '$project': {'_id': False, 'bcc_email': '$headers.Bcc', 'cc_email': '$headers.Cc',
#                      'bcc_name': '$headers.X-bcc', 'cc_name': '$headers.X-cc'}
#     }
#     group = {
#         '$group': {'_id': '$cc_email', 'names': {'$push': '$cc_name'}}
#     }

### 4)

Write a function with parameter subject, which gets all emails in a thread with that parameter, and orders them by date (ascending). “An email thread is an email message that includes a running list of all the succeeding replies starting with the original email.”, check for detail descriptions at https://www.techopedia.com/definition/1503/email-thread [4 points]

In [45]:
def get_emails_in_thread(collection, subject):
    """
    :param collection A PyMongo collection object
    :return All emails in the thread with that subject
    """    
    # YOUR CODE HERE
    # Find emails with that subject
    match = {
        '$match' : { 'headers.Subject' : {'$regex': subject}}
    }
    
    # Create datetime field for sorting using $dateFromString in MongoDB
    add = {
        '$addFields' : {'datetime':  {'$dateFromString': {
                                        'dateString': {
                                            '$substr': ['$headers.Date', 0, {'$subtract': [{'$strLenCP': '$headers.Date'}, 5]}]
                                        }}}
                       }
    }
    
    # Sort by date ascending
    sort = {
        '$sort' : {'datetime': pymongo.ASCENDING}
    }

    # keep relevant details
    project = {
        '$project' : {'_id':False, 'datetime': False}
    }
    
    if subject == '':
        raise Exception('Please enter a subject')
    else:
        cursor = collection.aggregate([match, add, sort, project])
        return list(cursor)

### 5)

Write a function which returns the percentage of emails sent on a weekend (i.e., Saturday and Sunday) as a `float` between 0 and 1. [6 points]

In [46]:
def get_percentage_sent_on_weekend(collection):
    """
    :param collection A PyMongo collection object
    :return A float between 0 and 1
    """    
    # YOUR CODE HERE
    weekends = collection.count_documents({'headers.Date': {'$regex' : 'Sat|Sun'}})
    all = collection.count_documents({})
    
    return weekends / float(all)

### 6)

Write a function with parameter limit. The function should return for each email account: the number of emails sent, the number of emails received, and the total number of emails (sent and received). Use the following format: [{"contact": "michael.simmons@enron.com", "from": 42, "to": 92, "total": 134}] and the information contained in the To, From, and Cc headers. Sort the output in descending order by the total number of emails. Use the parameter limit to specify the number of results to be returned. If limit is null, the function should return all results. If limit is higher than null, the function should return the number of results specified as limit. limit cannot take negative values. [10 points]

In [47]:
def get_emails_between_contacts(collection, limit):
    """
    Shows the communications between contacts
    Sort by the descending order of total emails using the To, From, and Cc headers.
    :param `collection` A PyMongo collection object    
    :param `limit` An integer specifying the amount to display, or
    if null will display all outputs
    :return A list of objects of the form:
    [{
        'contact': <<Another email address>>
        'from': 
        'to': 
        'total': 
    },{.....}]
    """
    # Count the From emails
    group_from = {
        '$group' : {'_id' : '$headers.From', 'from_count': {'$sum': 1}}
    }
    
    # Count the To emails which are not null and split them in mongoDB
    match_to = {
        '$match': {'headers.To': {'$ne': None}}
    }
    
    group_to = {
        '$group' : {'_id': '$headers.To', 'count': {'$sum': 1}}
    }
        
    # Count the Cc emails which are not null and split them in mongoDB
    match_cc = {
        '$match': {'headers.Cc': {'$ne': None}}
    } 
    
    group_cc = {
        '$group' : {'_id': '$headers.Cc', 'count': {'$sum': 1}}
    }
    
    # split, unwind and group operations for To and Cc emails (and trim the white space)
    split = {
        '$project': {'_id': False, 'count': True, 'split_emails': {'$split': ['$_id', ',']}}
    }
    
    unwind = {
        '$unwind': '$split_emails'
    }
    
    group_to_all = {
        '$group': {'_id': {'$trim': {'input': '$split_emails'}}, 'to_count': {'$sum': '$count'}}
    }
    
    group_cc_all = {
        '$group': {'_id': {'$trim': {'input': '$split_emails'}}, 'cc_count': {'$sum': '$count'}}
    }
    
    
    # facet and project the results together
    facet = {
        '$facet': {
            'from': [group_from], 
            'to': [match_to, group_to, split, unwind, group_to_all], 
            'cc': [match_cc, group_cc, split, unwind, group_cc_all]}
    }

    project = {
        '$project': {'_id': False, 'all': {'$setUnion': ['$cc', '$from', '$to']}}
    }
    
    unwind_all = {
        '$unwind': '$all'
    }
    
    replaceroot = {
        '$replaceRoot': {'newRoot': '$all'}
    }
    
    # group all the results by email address and do counting
    group_all = {
        '$group': {'_id': '$_id', 'from': {'$sum': '$from_count'}, 'to': {'$sum': '$to_count'}, 'cc': {'$sum': '$cc_count'}}
    }
    
    # add the cc counts to the to counts as report as To
    project_final = {
        '$project': {'_id':False, 'contact': '$_id', 'from': True, 'to': {'$sum': ['$to', '$cc']}, 'total': {'$sum': ['$from', '$to', '$cc']}}
    }
    
    sort = {
        '$sort': {'total': pymongo.DESCENDING}
    }
    
    limitBy = {
        '$limit': limit
    }
        
    # YOUR CODE HERE
    if limit < 0 and limit is not None:
        raise Exception('Limit cannot be negative')
    else:
        merge = None
        if limit is None:
            merge = collection.aggregate([facet, project, unwind_all, replaceroot, group_all, project_final, sort])
        else:
            merge = collection.aggregate([facet, project, unwind_all, replaceroot, group_all, project_final, sort, limitBy])
        return list(merge)

### 7)
Write a function to find out the number of senders who were also direct receivers. Direct receiver means the email is sent to the person directly, not via cc or bcc. [4 points]

In [48]:
def get_from_to_people(collection):
    """
    :param collection A PyMongo collection object
    :return the NUMBER of the people who have sent emails and received emails as direct receivers.
    """    
    # YOUR CODE HERE
    # Find all the distinct emails in headers.From
    group_from = {
        '$group' : {'_id' : '$headers.From'}
    }
    
    # Find all the distinct emails in headers.To which are not empty
    match_to = {
        '$match': { 'headers.To': {'$ne': None}}
    } 
    
    group_to = {
        '$group' : {'_id' : '$headers.To'}
    }
    
    from_emails = collection.aggregate([group_from])
    to_emails = collection.aggregate([match_to, group_to])
    
    # remove the white spaces before or after the emails
    send = [email['_id'].strip() for email in from_emails]
    receive = [email.strip() for emails in to_emails for email in emails['_id'].split(',')]
    
    return len(set(receive).intersection(send))

### 8)
Write a function with parameters start_date and end_date, which returns the number of email messages that have been sent between those specified dates, including start_date and end_date [4 points] 

In [49]:
def get_emails_between_dates(collection, start_date, end_date):
    """
    :param collection A PyMongo collection object
    :return All emails between the specified start_date and end_date
    """    
    # YOUR CODE HERE    
    # Python DateTime Object: start_date, end_date
    # Format of the date in MongoDB: Tue, 14 Nov 2000 04:17:00 -0800 (PST) [36 or 37 characters]
    # one outlier of the dates: Monday, March 12
    
    # group the same dates together
    group = {
        '$group': {'_id': '$headers.Date', 'count': {'$sum': 1}}
    }
    
    # make sure the string in headers.Date is in the right format, otherwise ignore the row
    # then project a new field using $dateFromString which gets a datetime object in mongoDB
    project = {
        '$project' : {'_id':False, 'count': True,
                          'datetime': {
                              '$cond': [{'$or': [{'$eq': [{'$strLenCP': '$_id'}, 36]}, 
                                                 {'$eq': [{'$strLenCP': '$_id'}, 37]}
                                                 ]},
                                        {'$dateFromString': {
                                            'dateString': {
                                                '$substr': ['$_id', 0, {'$subtract': [{'$strLenCP': '$_id'}, 5]}]
                                                }
                                            }
                                        }, 
                                        None]
                          }
                     }
    }
    
    # find the emails which are between the start date and end date included
    match = {
        '$match': {'$and': [{'datetime': {'$gte': start_date}},
                            {'datetime': {'$lte': end_date}}
                            ]}
    }
    
    # count them
    count = {
        '$group' : {'_id': None, 'sum': {'$sum': '$count'}}
    }
    
    num_emails = collection.aggregate([group, project, match, count])
    return num_emails.next()['sum']

## Task 2
This task will assess your ability to use the Hadoop Streaming API and MapReduce to process data. For each of the questions below, you are expected to write two python scripts, one for the Map phase and one for the Reduce phase. You are also expected to provide the correct parameters to the `hadoop` command to run the MapReduce process. Write down your answers in the specified cells below.

To get started, you need to download and unzip the YouTube dataset (available at http://edshare.soton.ac.uk/19547/) onto the machine where you have Hadoop installed (this should be the virtual machine provided).

To help you, `%%writefile` has been added to the top of the cells, automatically writing them to "mapper.py" and "reducer.py" respectively when the cells are run.

### 1) 
Using Youtube01-Psy.csv, find the hourly interval in which most spam was sent. The output should be in the form of a single key-value pair, where the value is a datetime at the start of the hour with the highest number of spam comments. [9 points]

In [50]:
%%writefile mapper.py
#!/usr/bin/env python
#Answer for mapper.py

# Pass all the hour in the form of (key, value) pair
# assign all values to have the same key so they go to the same reducer
import csv
import sys

lines = sys.stdin.readlines()
csvreader = csv.reader(lines)
headers = next(csvreader)

for row in csvreader:
    try:  # is a time in the format of YYYY-MM-DDTHH:MM:SS
        hour = row[2][0:14] + "00:00"
        print('{}\t{}'.format('hour_with_most_spam', hour))
    except:
        continue

Overwriting mapper.py


In [51]:
%%writefile reducer.py
#!/usr/bin/env python
#Answer for reducer.py
import sys
from collections import defaultdict

input_pairs = sys.stdin.readlines()

accumulator = defaultdict(lambda: 0)

for time in input_pairs:
    key_value = time.split('\t', 1)
    if len(key_value) != 2:
        continue
    time = str(key_value[1]).strip()
    accumulator[time] += 1

hour_with_most_spam = max(accumulator.iteritems(), key = lambda k: k[1])
print('hour_with_most_spam\t"{}"'.format(hour_with_most_spam[0]))

Overwriting reducer.py


In [54]:
%%bash
#Hadoop command to run the map reduce.
rm -rf output
hadoop-standalone-mode.sh

hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-input Youtube01-Psy.csv \
-mapper ./mapper.py \
-reducer ./reducer.py \
-output output \

-bash: line 3: hadoop-standalone-mode.sh: command not found
-bash: line 5: hadoop: command not found


In [55]:
#Expected key-value output format:
#hour_with_most_spam	"2013-11-10T10:00:00"

#Additional key-value pairs are acceptable, as long as the hour_with_most_spam pair is correct.

### 2) 
Find all comments associated with a username (the AUTHOR field). Return a JSON array of all comments associated with that username. (This should use the data from all 5 data files: Psy, KatyPerry, LMFAO, Eminem, Shakira) [11 points]

In [56]:
%%writefile mapper.py
#!/usr/bin/env python
#Answer for mapper.py

# Argument username is passed in the bash commands
# See two cells below
import csv
import sys

lines = sys.stdin.readlines()
csvreader = csv.reader(lines)
# remove the headers row in csv
headers = next(csvreader)

# the username are the command line arguments separated by whitespaces
username = ' '.join(sys.argv[1:])

for row in csvreader:
    # if username is not provided, we output all key,value pairs of all authors
    # if username == author, then we output key,value pairs of the same author
    author = row[1].strip()
    comment = row[3].strip()
    if username == "":
        print('{}\t{}'.format(author, comment))
    elif author == username:
        print('{}\t{}'.format(username, comment))


Overwriting mapper.py


In [57]:
%%writefile reducer.py
#!/usr/bin/env python
#Answer for reducer.py
import sys
import json
from collections import defaultdict

input_pairs = sys.stdin.readlines()

accumulator = defaultdict(list)

for row in input_pairs:
    key_value = row.split('\t', 1)
    if len(key_value) != 2:
        continue
    author = key_value[0]
    comment = key_value[1]
    accumulator[author].append(comment)

for (key, value) in accumulator.iteritems():
     print('{}\t{}'.format(key, json.dumps(value)))


Overwriting reducer.py


In [58]:
%%bash
# Enter the username you want to search
## set username as "" to find all username's list of comments
USERNAME="Corey Wilson"

# set the mapper to pass the USERNAME argument
MAPPER="./mapper.py $USERNAME"
# echo $MAPPER

#Hadoop command to run the map reduce.
rm -rf output
hadoop-standalone-mode.sh

hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-input Youtube01-Psy.csv,Youtube02-KatyPerry.csv,Youtube03-LMFAO.csv,Youtube04-Eminem.csv,Youtube05-Shakira.csv \
-mapper "$MAPPER" \
-reducer ./reducer.py \
-output output

-bash: line 11: hadoop-standalone-mode.sh: command not found
-bash: line 13: hadoop: command not found


In [ ]:
#Expected key-value output format:
#John Smith	["Comment 1", "Comment 2", "Comment 3", "etc."]
#Jane Doe	["Comment 1", "Comment 2", "Comment 3", "etc."]